In [2]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Import Data

In [3]:
interactions_df = pd.read_csv("full_interactions_dataset.csv")

In [4]:
df = pd.read_csv("full_dataset.csv")

/Users/melissamunz/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.head()

,Unnamed: 0,timestamp_x,eventType_x,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,...,text,lang,timestamp_y,eventType_y,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
0,3,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,...,All of this work is still very early. The firs...,en,1459270471,VIEW,2873028073541627603,-3228450896145227905,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,SP,BR,1.0
1,4,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,...,The alarm clock wakes me at 8:00 with stream o...,en,1459539433,VIEW,8414731042150985013,4543899740167763020,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,1.0
2,5,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,...,We're excited to share the Google Data Center ...,en,1459476271,VIEW,-7267769888748948232,-1350132153769633851,Android - Native Mobile App,MG,BR,1.0
3,6,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,...,We're excited to share the Google Data Center ...,en,1459476266,VIEW,-7267769888748948232,-1350132153769633851,Android - Native Mobile App,MG,BR,1.0
4,7,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,...,We're excited to share the Google Data Center ...,en,1459476139,LIKE,-7267769888748948232,-1350132153769633851,NaN,NaN,NaN,2.0


# Split the Data in Test and Train Sets

In [10]:
X = None
y = None

In [23]:
interactions_train, interactions_test = train_test_split(interactions_df, test_size = 0.2, random_state = 42)

In [24]:
interactions_train_df = pd.to_frame(data=interactions_train)

AttributeError: module 'pandas' has no attribute 'to_frame'

In [25]:
print(f'Length of the training set is: {len(interactions_train)}')
print(f'Length of the testing set is: {len(interactions_test)}')

Length of the training set is: 31260
Length of the testing set is: 7815


# Evaluation

In Recommender Systems, there are a set metrics commonly used for evaluation. We chose to work with Top-N accuracy metrics, which evaluates the accuracy of the top recommendations provided to a user, comparing to the items the user has actually interacted in test set.

In [26]:
#Indexing by person Id to speed up searches

indexed_interactions_df = interactions_df.set_index('personId')

indexed_interactions_train = interactions_train.set_index('personId')

indexed_interactions_test = interactions_test.set_index('personId')

In [27]:
#Merge data with movie information

def get_interactions(personId, interactions_df):
    items = interactions_df.loc[personId]['contentId']
    return set(items if type(items) == pd.Series else [items])
    

In [ ]:
interacted_items = interactions_df.loc[person_id]['contentId']

    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])